# Study Outcome Analysis

## Notebook Summary
- Load validated experiment results for all participants.
- Compute success rates for correctly reported result numbers by condition and in total.
- Compute success rates for correctly reported result layers by condition and in total.
- Generate descriptive statistics with confidence intervals and export CSV summaries.


## Imports and Helper Functions
Load common imports, paths, condition order, and statistics helper functions from the shared notebook.


In [2]:
# Load shared imports, constants, and helper utilities from the common notebook.
%run functions.ipynb


## Load Validated Experiment Data
Read the cleaned experiment-level results dataset that is used for outcome statistics.


In [3]:
# Load validated experiment results (training trials are already excluded in this export).
data_complete = pd.read_csv(rf'{export_data}results_experiment_valid.csv', sep=";")

# Preview the input table before deriving outcome metrics.
display(data_complete)


,BlockId,CommitResultDate,Condition,EndInteractionDate,ExpectedResultLayer,ExpectedResultNumber,LayerNumberConfiguration,ProbandId,ProvidedResultLayer,ProvidedResultNumber,...,TrialNumber,CleanedUpInteractionStart_DT,CleanedUpInteractionEnd_DT,ResultNumberDifference,ResultLayerDifference,ResultLayerCorrect,ResultNumberCorrect,DurationMS,DurationMS_Cleaned,countCondition
0,0,2022-08-01T14:15:22.795Z,No Feedback,2022-08-01T14:15:19.119Z,5,166,"[136, 147, 130, 139, 166, 150, 132]",2,5,166,...,0,2022-08-01 14:14:05.504,2022-08-01 14:15:18.527,0,0,True,True,78.538,73.023,2
1,0,2022-08-01T14:16:08.915Z,No Feedback,2022-08-01T14:16:03.467Z,3,165,"[122, 147, 165, 150, 129, 138, 149]",2,3,165,...,1,2022-08-01 14:15:26.143,2022-08-01 14:16:00.890,0,0,True,True,40.671,34.747,3
2,0,2022-08-01T14:17:10.611Z,No Feedback,2022-08-01T14:17:06.246Z,5,154,"[148, 130, 142, 149, 154, 131, 147]",2,4,154,...,2,2022-08-01 14:16:10.883,2022-08-01 14:17:05.397,0,-1,False,True,57.331,54.514,4
3,0,2022-08-01T14:18:09.619Z,No Feedback,2022-08-01T14:18:05.670Z,3,164,"[141, 132, 164, 110, 152, 147, 154]",2,3,164,...,3,2022-08-01 14:17:11.970,2022-08-01 14:18:04.603,0,0,True,True,55.058,52.633,5
4,0,2022-08-01T14:19:06.811Z,No Feedback,2022-08-01T14:19:02.997Z,7,154,"[142, 148, 140, 143, 144, 132, 154]",2,7,154,...,4,2022-08-01 14:18:11.962,2022-08-01 14:19:02.010,0,0,True,True,53.376,50.048,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,3,2022-08-23T11:27:13.692Z,No Feedback,2022-08-23T11:27:09.628Z,7,159,"[153, 122, 128, 144, 140, 154, 159]",23,7,159,...,79,2022-08-23 11:26:33.878,2022-08-23 11:27:08.564,0,0,True,True,37.272,34.686,501
1760,3,2022-08-23T11:27:49.516Z,No Feedback,2022-08-23T11:27:45.172Z,5,157,"[146, 142, 133, 156, 157, 128, 138]",23,5,157,...,80,2022-08-23 11:27:15.467,2022-08-23 11:27:44.655,0,0,True,True,31.479,29.188,502
1761,3,2022-08-23T11:28:51.795Z,No Feedback,2022-08-23T11:28:46.148Z,6,148,"[139, 140, 143, 147, 142, 148, 141]",23,6,148,...,81,2022-08-23 11:27:50.891,2022-08-23 11:28:45.618,0,0,True,True,56.632,54.727,503
1762,3,2022-08-23T11:29:32.908Z,No Feedback,2022-08-23T11:29:29.764Z,1,152,"[152, 139, 138, 134, 145, 144, 148]",23,1,152,...,82,2022-08-23 11:28:54.105,2022-08-23 11:29:28.940,0,0,True,True,37.968,34.835,504


## Success Rates For Result Numbers
Compute participant-level success rates for correct result numbers, summarize by condition and total, and export statistics.


In [17]:
# Compute participant-level success rates for correctly reported result numbers per condition.
sucess_rates = data_complete[['ProbandId', 'TrialId', 'Condition', 'ResultNumberCorrect' ]].groupby(['ProbandId', 'Condition'])['ResultNumberCorrect'].mean().reset_index(name="SuccessRate")

# Inspect participant-level success rates.
display(sucess_rates)

# Build grouped views for condition-level and overall participant summaries.
grouped = sucess_rates.groupby("Condition")["SuccessRate"]
grouped_totel = sucess_rates.groupby("ProbandId")["SuccessRate"].mean()

# Calculate descriptive statistics with configured percentiles.
desc = grouped.describe(percentiles=percentiles)
desc_total = grouped_totel.describe(percentiles=percentiles)

# Convert to DataFrames so confidence intervals can be appended.
stats = pd.DataFrame(desc)
stats_total = pd.DataFrame(desc_total).T

# Add confidence interval columns to both summary tables.
computeCI(stats)
computeCI(stats_total)

#computeWhiskers(grouped_totel.T, stats, condition_names)
#computeWhiskers(grouped_totel.T, stats_total, [])

# Merge condition-wise and total statistics into a single export table.
stats_complete = pd.concat([stats, stats_total], ignore_index=False)

# Preview and export the final statistics table for reported numbers.
display(stats_complete.T)

stats_complete.to_csv(rf'{export_data}outcome_result-number_stats.csv', sep=";", index=True)


,ProbandId,Condition,SuccessRate
0,2,Combined Feedback,0.952381
1,2,No Feedback,1.000000
2,2,Tactile Feedback,0.952381
3,2,Visual Feedback,0.952381
4,4,Combined Feedback,1.000000
...,...,...,...
79,22,Visual Feedback,1.000000
80,23,Combined Feedback,0.904762
81,23,No Feedback,0.904762
82,23,Tactile Feedback,0.809524


,Combined Feedback,No Feedback,Tactile Feedback,Visual Feedback,SuccessRate
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.977324,0.947846,0.927438,0.965986,0.954649
std,0.035695,0.060054,0.089871,0.052474,0.045740
min,0.904762,0.809524,0.714286,0.809524,0.857143
5%,0.904762,0.857143,0.761905,0.857143,0.869048
10%,0.904762,0.857143,0.809524,0.904762,0.880952
25%,0.952381,0.904762,0.904762,0.952381,0.952381
50%,1.000000,0.952381,0.952381,1.000000,0.964286
75%,1.000000,1.000000,1.000000,1.000000,0.988095
90%,1.000000,1.000000,1.000000,1.000000,1.000000


## Success Rates For Result Layers
Repeat the same outcome pipeline for correct result layers and export the descriptive statistics table.


In [18]:
# Compute participant-level success rates for correctly reported result layers per condition.
sucess_rates = data_complete[['ProbandId', 'TrialId', 'Condition', 'ResultLayerCorrect' ]].groupby(['ProbandId', 'Condition'])['ResultLayerCorrect'].mean().reset_index(name="SuccessRate")

# Inspect participant-level success rates.
display(sucess_rates)

# Build grouped views for condition-level and overall participant summaries.
grouped = sucess_rates.groupby("Condition")["SuccessRate"]
grouped_totel = sucess_rates.groupby("ProbandId")["SuccessRate"].mean()

# Calculate descriptive statistics with configured percentiles.
desc = grouped.describe(percentiles=percentiles)
desc_total = grouped_totel.describe(percentiles=percentiles)

# Convert to DataFrames so confidence intervals can be appended.
stats = pd.DataFrame(desc)
stats_total = pd.DataFrame(desc_total).T

# Add confidence interval columns to both summary tables.
computeCI(stats)
computeCI(stats_total)

#computeWhiskers(grouped_totel.T, stats, condition_names)
#computeWhiskers(grouped_totel.T, stats_total, [])

# Merge condition-wise and total statistics into a single export table.
stats_complete = pd.concat([stats, stats_total], ignore_index=False)

# Preview and export the final statistics table for reported layers.
display(stats_complete.T)

stats_complete.to_csv(rf'{export_data}outcome_result-layer_stats.csv', sep=";", index=True)


,ProbandId,Condition,SuccessRate
0,2,Combined Feedback,0.904762
1,2,No Feedback,0.952381
2,2,Tactile Feedback,0.904762
3,2,Visual Feedback,0.904762
4,4,Combined Feedback,1.000000
...,...,...,...
79,22,Visual Feedback,1.000000
80,23,Combined Feedback,0.714286
81,23,No Feedback,0.857143
82,23,Tactile Feedback,0.714286


,Combined Feedback,No Feedback,Tactile Feedback,Visual Feedback,SuccessRate
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.959184,0.931973,0.886621,0.963719,0.935374
std,0.071088,0.069939,0.142441,0.051957,0.064172
min,0.714286,0.714286,0.476190,0.809524,0.773810
5%,0.857143,0.857143,0.714286,0.904762,0.821429
10%,0.904762,0.857143,0.714286,0.904762,0.857143
25%,0.952381,0.904762,0.809524,0.952381,0.892857
50%,1.000000,0.952381,0.952381,1.000000,0.952381
75%,1.000000,1.000000,1.000000,1.000000,0.988095
90%,1.000000,1.000000,1.000000,1.000000,1.000000
